### 데이터 불러오기

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms

In [23]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 2048

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


### 데이터 일부 확인

In [24]:
import matplotlib.pyplot as plt
import numpy as np

# 이미지를 보여주기 위한 함수

def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()


# 학습용 이미지를 무작위로 가져오기
dataiter = iter(trainloader)
images, labels = dataiter.next()

# 이미지 보여주기
# imshow(torchvision.utils.make_grid(images))
# 정답(label) 출력
print(' '.join(f'{classes[labels[j]]:5s}' for j in range(batch_size)))

bird  horse ship  car   cat   plane dog   cat   dog   bird  cat   frog  dog   cat   dog   deer  deer  deer  truck truck truck plane frog  horse dog   cat   ship  bird  frog  plane cat   frog  bird  dog   cat   ship  car   bird  cat   frog  cat   deer  bird  car   horse horse plane dog   dog   truck truck car   horse bird  dog   deer  car   plane dog   plane frog  car   car   car   ship  dog   frog  horse deer  plane ship  frog  bird  cat   frog  cat   dog   car   truck dog   dog   frog  ship  dog   dog   bird  dog   frog  cat   frog  frog  car   truck horse horse ship  frog  horse ship  plane car   frog  cat   truck frog  horse frog  dog   horse truck car   horse ship  bird  ship  car   car   horse deer  car   deer  dog   dog   plane truck frog  frog  dog   ship  truck deer  frog  dog   cat   truck dog   dog   car   bird  plane ship  frog  truck frog  frog  horse plane truck dog   cat   dog   frog  plane cat   cat   frog  horse deer  deer  plane ship  plane truck deer  car   dog   car 

## 모델 정의
이후부터 과제 진행하시면 됩니다! 위는 수정 XX

'#####으로 표시된 부분에 작성하세요!

In [25]:
import torch.nn as nn

In [26]:
trainset.data.shape

(50000, 32, 32, 3)

In [33]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, ),           # -> batch, 16, 30, 30
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.Dropout2d(0.2),
            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3),          # -> batch, 32, 28, 28
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Dropout2d(0.2),
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3),          # -> batch, 64, 26, 26
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Dropout2d(0.2),
            nn.MaxPool2d(kernel_size=2,stride=2),                                # -> batch, 64, 13, 13

            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=2),           # -> batch, 64, 12, 12
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Dropout2d(0.2),
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=2),          # -> batch, 64, 11, 11
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Dropout2d(0.2),
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=2),          # -> batch, 64, 10, 10
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Dropout2d(0.2),
        )
        self.fc_layer = nn.Sequential(                                          
            nn.Linear(64*10*10,300),                                            
            nn.ReLU(),
            nn.Linear(300,10)                                                
        )  


    def forward(self, x):
        out = self.layer(x)
        out = out.view(out.size(0), -1)
        out = self.fc_layer(out)

        return out


net = Net()

In [34]:
use_cuda = True
if use_cuda and torch.cuda.is_available():
    net.cuda()

In [35]:
torch.cuda.is_available()

True

## 손실함수, optimizer 정의
필요한 모듈을 import하고 손실함수는 CrossEntropyLoss, optimizer는 SGD를 사용해주세요.

optimizer의 하이퍼파라미터는 lr=0.001, momentum=0.9로 통일합니다.

작성한 모델이 무거워 실행 시간이 오래 걸린다면 GPU를 활용하세요.

In [36]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
# optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
optimizer = optim.Adam(net.parameters(), lr=0.001, weight_decay=0.005)

## 모델 구조 확인
필요한 모듈을 import하여 모델의 구조를 확인하세요.

In [37]:
from torchsummary import summary
summary(net, (3, 32, 32), batch_size=4)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [4, 16, 30, 30]             448
       BatchNorm2d-2            [4, 16, 30, 30]              32
              ReLU-3            [4, 16, 30, 30]               0
         Dropout2d-4            [4, 16, 30, 30]               0
            Conv2d-5            [4, 32, 28, 28]           4,640
       BatchNorm2d-6            [4, 32, 28, 28]              64
              ReLU-7            [4, 32, 28, 28]               0
         Dropout2d-8            [4, 32, 28, 28]               0
            Conv2d-9            [4, 64, 26, 26]          18,496
      BatchNorm2d-10            [4, 64, 26, 26]             128
             ReLU-11            [4, 64, 26, 26]               0
        Dropout2d-12            [4, 64, 26, 26]               0
        MaxPool2d-13            [4, 64, 13, 13]               0
           Conv2d-14            [4, 64,

## Train

In [38]:
# 모델을 학습

for epoch in range(100):

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # [inputs, labels]의 목록인 data로부터 입력을 받은 후;
        inputs, labels = data
        inputs, labels = inputs.cuda(), labels.cuda()
        optimizer.zero_grad()
        y_pred = net(inputs)
        loss = criterion(y_pred, labels)
        loss.backward()
        optimizer.step()

        # 통계를 출력합니다.
        running_loss += loss.item()

    print(f'{epoch + 1} loss: {running_loss / batch_size:.3f}')
    running_loss = 0.0

print('Finished Training')

1 loss: 0.028
2 loss: 0.022
3 loss: 0.020
4 loss: 0.019
5 loss: 0.018
6 loss: 0.017
7 loss: 0.017
8 loss: 0.016
9 loss: 0.015
10 loss: 0.015
11 loss: 0.014
12 loss: 0.014
13 loss: 0.014
14 loss: 0.013
15 loss: 0.013
16 loss: 0.013
17 loss: 0.012
18 loss: 0.012
19 loss: 0.012
20 loss: 0.012
21 loss: 0.012
22 loss: 0.011
23 loss: 0.011
24 loss: 0.011
25 loss: 0.011
26 loss: 0.011
27 loss: 0.011
28 loss: 0.011
29 loss: 0.011
30 loss: 0.010
31 loss: 0.010
32 loss: 0.010
33 loss: 0.010
34 loss: 0.010
35 loss: 0.010
36 loss: 0.010
37 loss: 0.010
38 loss: 0.010
39 loss: 0.010
40 loss: 0.010
41 loss: 0.010
42 loss: 0.010
43 loss: 0.010
44 loss: 0.009
45 loss: 0.009
46 loss: 0.009
47 loss: 0.009
48 loss: 0.009
49 loss: 0.009
50 loss: 0.009
51 loss: 0.009
52 loss: 0.009
53 loss: 0.009
54 loss: 0.009
55 loss: 0.009
56 loss: 0.009
57 loss: 0.009
58 loss: 0.009
59 loss: 0.009
60 loss: 0.009
61 loss: 0.009
62 loss: 0.009
63 loss: 0.009
64 loss: 0.009
65 loss: 0.009
66 loss: 0.009
67 loss: 0.009
68 l

## Test

In [39]:
dataiter = iter(testloader)
images, labels = dataiter.next()

In [40]:
## 아래 코드 그대로 실행

correct = 0
total = 0
# 학습 중이 아니므로, 출력에 대한 변화도를 계산할 필요가 없습니다
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images, labels = images.cuda(), labels.cuda()
        # 신경망에 이미지를 통과시켜 출력을 계산합니다
        outputs = net(images)
        # 가장 높은 값(energy)를 갖는 분류(class)를 정답으로 선택하겠습니다
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct // total} %')

Accuracy of the network on the 10000 test images: 71 %


## 클래스별 accuracy 비교

In [41]:
## 아래 코드 그대로 실행

# 각 분류(class)에 대한 예측값 계산을 위해 준비
correct_pred = {classname: 0 for classname in classes}
total_pred = {classname: 0 for classname in classes}

# 변화도는 여전히 필요하지 않습니다
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images, labels = images.cuda(), labels.cuda()
        outputs = net(images)
        _, predictions = torch.max(outputs, 1)
        # 각 분류별로 올바른 예측 수를 모읍니다
        for label, prediction in zip(labels, predictions):
            if label == prediction:
                correct_pred[classes[label]] += 1
            total_pred[classes[label]] += 1


# 각 분류별 정확도(accuracy)를 출력합니다
for classname, correct_count in correct_pred.items():
    accuracy = 100 * float(correct_count) / total_pred[classname]
    print(f'Accuracy for class: {classname:5s} is {accuracy:.1f} %')

Accuracy for class: plane is 74.3 %
Accuracy for class: car   is 82.7 %
Accuracy for class: bird  is 56.5 %
Accuracy for class: cat   is 60.3 %
Accuracy for class: deer  is 66.0 %
Accuracy for class: dog   is 65.3 %
Accuracy for class: frog  is 78.7 %
Accuracy for class: horse is 65.6 %
Accuracy for class: ship  is 82.7 %
Accuracy for class: truck is 83.9 %
